In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

# Building dataset of finance faculty profiles

Author profiles used as features in our citation network. 

Note that the raw data was collected by customized web scraping for each source (each source required different versions of web scraping due to different html codes).

In [ ]:
# 1. From AFA Finance Directory

afa_fin_dir = pd.read_excel("faculty profile data/AFA_FIN_DIR.xlsx")
afa_fin_dir2 = pd.read_excel("faculty profile data/AFA_FIN_DIR_2000_1950.xlsx")

# Clean data
afa_fin_dir['First Name'] = afa_fin_dir[0].apply(lambda x: ' '.join(x.split()[:1]))

def check_even(number):
    if len(number.split())>=3:
        return ' '.join(number.split()[1:-1])
    else:
        return np.nan
    
afa_fin_dir['Middle Name'] = afa_fin_dir[0].apply(check_even)
afa = pd.concat([afa_fin_dir,afa_fin_dir2])

def remove_parentheses(content):
    """
    Function to remove parentheses and any content inside them from a string.
    """
    # Using regex to remove content in parentheses
    return re.sub(r"\s*\([^)]*\)", "", content)

afa['Authors'] = afa[0].apply(lambda x: remove_parentheses(x) if isinstance(x, str) else x)
afa['Source'] = 'AFA'
afa.rename(columns={1:'University'}, inplace=True)

In [ ]:
# 2. From EFA

efa = pd.read_excel("faculty profile data/EFA.xlsx",index_col = 0)
efa['Source'] = 'EFA'

In [ ]:
# 3. From AEFIN 

aefin = pd.read_excel("faculty profile data/aefin.xlsx",index_col = 0)
aefin['Source'] = 'AEFIN'

afa2 = afa[aefin.columns]

In [ ]:
# 4. From ABFER

abfer = pd.read_excel("faculty profile data/ABFER.xlsx",index_col = 0)
abfer['Source'] = 'ABFER'

In [ ]:
# 5. From NBER 

NBER = pd.read_excel("faculty profile data/NBER.xlsx",index_col = 0)
NBER['Source'] = 'NBER'

In [ ]:
# 6. From CEPR 

cepr = pd.read_excel("faculty profile data/profile_cepr.xlsx",index_col = 0)
cepr['Source'] = 'cepr'
cepr.columns = NBER.columns # Unify column name across data from different sources

In [ ]:
# 7. From IZA
 
iza = pd.read_excel("faculty profile data/profile_iza.xlsx",index_col = 0)
iza['Source'] = 'iza'
iza1 = iza[[0,2,'Source']]
iza1.columns = iza1.columns

In [ ]:
# Combine these separate author profile datasets

oth = pd.concat([afa2, efa,aefin,abfer,NBER,cepr]).drop_duplicates()

# Data cleaning
oth['Authors'] = oth['Authors'].apply(lambda x: remove_parentheses(x) if isinstance(x, str) else x)
oth['Last Name'] = oth['Authors'].apply(lambda x: ' '.join(x.split()[-1:]))
oth['First Name'] = oth['Authors'].apply(lambda x: ' '.join(x.split()[:1]))
oth['Middle Name'] = oth['Authors'].apply(check_even)
oth = oth.applymap(lambda x: x.capitalize() if isinstance(x, str) else x)
oth['University'] = oth['University'].apply(lambda x: remove_parentheses(x) if isinstance(x, str) else x)

In [ ]:
# Identify country of associated university
def get_country_from_university_comprehensive(university_name):
    """
    Comprehensive function to determine the country of a university based on its name.
    This version includes a broader range of well-known universities and enhanced inference logic.
    """
    # Check if the university name is not a string (e.g., NaN)
    if not isinstance(university_name, str):
        return "Unknown"

    # Expanded list of specific well-known universities globally
    specific_universities = {
        "Yale University": "USA",
        "York University": "Canada",
        "Yale": "USA",
        "Western university": "Canada",
        "Vanderbilt University": "USA",
        "Wellesley College":"USA",
        "Williams college":"USA",
        "Wake forest university":"USA",
        "Harvard University": "USA",
        "Stanford University": "USA",
        "Princeton University": "USA",
        "Massachusetts Institute of Technology": "USA",
        "University of Cambridge": "UK",
        "University of Oxford": "UK",
        "Imperial College London": "UK",
        "University College London": "UK",
        "London School of Economics": "UK",
        "London Business School": "UK",
        "University of Toronto": "Canada",
        "McGill University": "Canada",
        "University of British Columbia": "Canada",
        "University of Melbourne": "Australia",
        "University of Sydney": "Australia",
        "Australian National University": "Australia",
        "University of Hong Kong": "Hong Kong",
        "Hong Kong Polytechnic University": "Hong Kong",
        "National University of Singapore": "Singapore",
        "Tsinghua University": "China",
        "Peking University": "China",
        "University of Tokyo": "Japan",
        "Babson College": "USA",
        "Northwestern University": "USA",
        "Columbia University": "USA",
        "Boston university": "USA",
        "Dartmouth college": "USA",
        "Duke university": "USA",
        "Brown university": "USA",
        "Cornell university": "USA",
        "Emory university": "USA",
        "Boston college": "USA",
        "georgetown university": "USA",
        "Tufts university": "USA",
        "university of notre dame": "USA",
        "Johns hopkins university": "USA",
        "University of rochester": "USA",
        "Rutgers university": "USA",
        "Carnegie mellon university": "USA",
        "case western reserve universit": "USA",
        "university of houston": "USA",
        "northeastern university": "USA",
        "federal reserve": "USA",
        "nanyang technological university": "Singapore",
        "singapore management university": "Singapore",
        "baylor university": "USA",
        "cuny-baruch college": "Italy",
        "bocconi": "Italy",
        "auburn university": "USA",
        "brigham young university": "USA",
        "depaul university": "USA",
        "covenant university": "Nigeria",
        "southern methodist university": "USA",
        "insead": "France",
        "temple university": "USA",
        "securities and exchange commission": "USA",
        "old dominion university": "USA",
        "st. louis university": "USA",
        "cornerstone research": "USA",
        "rice university": "USA",
        "tel aviv university": "Israel",
        "clemson university": "USA",
        "icfai university": "India",
        "purdue": "USA",
        "claremont mckenna college": "USA",
        "Cnmv": "Spain",
        "university of lausanne": "Switzerland",
        "massey university": "New Zealand",
        "Instituto de empresa": "Spain",
        "wayne state university": "USA",
        "university of warwick": "UK",
        "brandeis": "USA",
        "syracuse uni": "USA",
        "goldman, sachs": "USA",
        "cheung kong graduate school of business": "China",
        "college of william and mary": "USA",
        "erasmus university": "Netherlands",
        "tulane ": "USA",
        "stevens institute of technology": "USA",
        "loyola marymount university": "USA",
        "wilfrid laurier university": "Canada",
        "university of waterloo": "Canada",
        "mcmaster uni": "Canada",
        "fudan uni": "China",
        "lancaster uni": "UK",
        "compass lexecon": "USA",
        "world bank": "USA",
        "james madison univ": "USA",
        "Cemfi": "Spain",
        "drexel universit": "USA",
        "iese business scho": "Spain",
        "new economic school":"Russia",
        "j.p. morgan": "USA",
        "credit suisse": "USA",
        "edhec business scho": "France",
        "marquette uni": "USA",
        "Lehigh university": "USA",
        "George mason unive": "USA",
        "pace univers": "USA",
        "32 advisiors, financial services": "USA",
        "swedish house of finance": "Sweden",
        "swinburne university of technology": "Australia",
        "t-mobile": "USA",
        "talladega college": "USA",
        "tamkang uni": "Taiwan",
        "tashkent state institute of finance": "Uzbekistan",
        "technical university of crete": "Greece",
        "stony brook unive": "USA",
        "tilburg university": "Netherlands",
        "dimensional fund advis": "USA",
        "simon fraser univ": "Canada",
        "suffolk univers": "USA",
        "monmouth univer": "USA",
        "blackrock": "USA",
        "villanova univers": "USA",
        "tribhuvan unive": "Nepal",
        "bloomberg": "USA",
        "moody's": "USA",
        "hofstra unive": "USA",
        "st. mary's uni": "USA",
        "st. lawrence uni": "USA",
        "st. olaf colle": "USA",
        "st louis univer": "USA",
        "stanford": "USA",
        "pontificia comillas": "Spain",
        "national central univer": "Taiwan",
        "monash unive": "Australia",
        "birla institute of management techn": "India",
        "pompeu fabra": "Spain",
        "fannie mae": "USA",
        "john carroll uni": "USA",
        "st. ambrose unive": "USA", "state corporation commission va": "USA", 
        "sukkur iba universit": "Pakistan", "Amherst": "USA",
        "fundacao getulio varg": "Brazil", "cuny-hunter coll": "USA", 
        "susquehanna unive": "USA","sri sringeri sharada institute of managem": "India",
        "sree narayana guru": "India", "td ameritrade": "USA",
        "bentley colleg": "USA", "ben-gurion universi": "Israel",
        "eafit universi": "Columbia", "universite côte d'azur": "France", 
        "technische universitaet wien": "Austria",
        "sp jain school of global managem": "Dubai",
         "seton hall universit": "USA", "pondicherry uni": "India",
         "esade business scho": "Spain", "Chapman university": "USA",
         "national university": "USA", "bond univers": "Queensland",
          "morgan stanley": "USA", "southern university of science and technolog": "China",
        "southern baptist univ": "USA", "société générale corporate & investment bank": "France",
         "walden universi": "USA", "creighton univ": "USA",
         "university of reading": "UK", "iesa escuela de gerencia": "Dominican Republic",
          "Esade": "Spain", "berkeley research group": "USA",
          "essec business school": "France", "bharathidasan unive": "India",
          "king fahd university of petroleum and minerals": "Saudi Arabia", 
          "skema business scho": "France",
           "siena colleg": "USA", "siksha o anusandhan unive": "India",
        "eth-zentrum": "Switzerland", "laval university": "Canada",
         "aon hewitt investment consultin": "USA", "whu, otto beisheim": "Germany",
        "universidad adolfo ibanez": "Chile", "comsats institute of information techno": "Pakistan",
        "citadel": "USA", "the brattle group": "USA",
        "the abacus group llc": "USA", "technology by bilan": "Switzerland",
        "Nber": "USA", "queens univ": "Canada",
        "finra": "USA", "Peterson institute for international economi": "USA",
        "university of the cumberlands": "USA", "university of toledo": "USA",
        "clark atlanta universit": "USA", "universidad pablo de olavide": "Spain",
        "menlo colle": "USA", "xavier univ": "USA", "imd international": "Switzerland",
        "international monetary fund": "USA", "vrije universiteit": "Netherlands", 
        "itesm": "Mexico", "qs investors, llc": "USA", "concordia university": "Canada",
        "bilkent university": "Turkey", "kdi school of public policy and management": "Korea", 
        "pepperdine university": "USA", "Cunef": "Spain", 
        "bahria university": "Pakistan",  "hosei university": "Japan",  
        "thunderbird school of global manage": "USA",  "anadolu university": "Turkey", 
         "grenoble ecole de management": "France",  "university of auckland": "New Zealand", 
         "university of exeter": "UK",  "fairleigh dickinson university": "USA", 
         "universidad de los andes": "Columbia",  "griffith universit": "Australia", 
         "university of exeter": "UK",  "essect": "Tunisia", 
         "fordham university": "USA",  "Universidad cardenal herrera-ceu": "Spain", 
         "hanken school of economics": "Finland",  "lal bahadur shastri institute of management": "India", 
         "macquarie universit": "Australia",  "ada university": "Azerbaijan", 
         "dalhousie university": "Canada",  "Universidad de navarra": "Spain", 
         "aalto university": "Finland",  "aarhus university": "Denmark", 
         "groupe hautes etudes commerciales": "France", 
         "iim ahmedabad, icai, icwai, icsi": "India","institut d'administration des enterprisesaix": "France",
         "insper institute of education and research": "Brazil","ing investment man": "Netherlands",
         "ilma university, academics": "Pakistan","howard univers": "USA",
         "infosys bpo": "India","independent university, department of finance": "Bangladesh","imf": "USA","finra": "USA",
        "iim lucknow": "India","lancaster": "UK",
        "charles schwab": "USA", "clark unive": "USA",

        # More specific universities can be added here
    }

    # Enhanced inference logic
    country_inference = {
        "USA": ['abilene', 'houston','buffalo',"new jersey","baltimore","long island","saint","ilinois", "st. ", "memphis","state street", "richmond","fairfield", "santa clara","cincinnati","america", "austin", "state uni", "louisville","oakland", "sydney", "pittsburgh", "san diego", "Miami","Chicago", "Texas", "California", "Florida", "New York", "Illinois", "Pennsylvania", "Ohio", "Michigan", "Georgia", "North Carolina", "Massachusetts", "Virginia", "Washington", "Indiana", "Arizona", "Tennessee", "Missouri", "Maryland", "Wisconsin", "Minnesota", "Colorado", "Alabama", "South Carolina", "Louisiana", "Kentucky", "Oregon", "Oklahoma", "Connecticut", "Iowa", "Mississippi", "Arkansas", "Kansas", "Utah", "Nevada", "New Mexico", "West Virginia", "Nebraska", "Idaho", "Hawaii", "Maine", "New Hampshire", "Rhode Island", "Montana", "Delaware", "South Dakota", "North Dakota", "Alaska", "Vermont", "Wyoming"],
        "Canada": ["montréal","canada","ottawa", "calgary","montréal","Ontario", "Quebec", "British Columbia", "Alberta", "Manitoba", "Saskatchewan", "Nova Scotia", "New Brunswick", "Newfoundland and Labrador", "Prince Edward Island"],
        "Australia": ["australia","wollongong","Queensland", "New South Wales", "Victoria", "South Australia", "Western Australia", "Tasmania"],
        "UK": ["essex", "british","London", "Cambridge", "Oxford", "Edinburgh", "Glasgow", "Manchester", "Bristol", "Birmingham", "Liverpool", "Leeds", "Sheffield", "Nottingham", "Newcastle", "Leicester", "Southampton", "Brighton"],
        "Sweden": ["swedish","Stockholm",'sveriges riksbank'],
        "New Zealand": ['auckland',"otago"],
        'United Arab Emirates':['dubai','uae','abu dhabi'], 
        "Taiwan": ["taiwan"],
        'Pakistan':['Pakistan'],
        'Rome':['rome'],
        "Ireland": ["Dublin"],
        'Finland': ['turku'],
        'Italy': ['milano'],
        "Singapore": ["Singapore"],
        "Portugal": ["lisbon","portuguesa",'lisboa'],
        "Tunisia": ['tunis'],
        "Cyprus": ["cyprus"],
        "Germany": ["munich","dresden",'frankfurt','mannheim'],
        "Israel": ["haifa",'jerusalem'],
        "Chile": ["chile"],
        "Mexico": ["méxico","monterrey"],
        "Greece": ['athens'],
        "Norway": ["norwegian"],
        "Austria": ["Vienna"],
        "Oman": ["Oman"],
        "Spain": ['granada',"españa","madrid","deusto"],
        "Korea": ['hongik',"unist","sogang", "yonsei","sung kyun kwan","chung-ang","korea"],
        "Hong Kong": ["hong kong"],
        "Netherlands": ["netherlands","amsterdam",'groningen','maastricht'],
        "Switzerland": ["swiss","zurich"],
        "India": ["india","tiruchirapalli","delhi",'rourkela'],
        "Denmark": ["copenhagen"],
        "France": ["Paris","Marseille","Lyon","Toulouse","Nice", "Nantes","Strasbourg","Montpellier", "Bordeaux", "Lille"],
        "China": ["wenzhou","tsing hua", "shandong","sichuan", "macau", "Peking", "Beijing","Shanghai","Guangzhou","Shenzhen","Chengdu","Xi'an","Wuhan","Chongqing","Tianjin"]       
        # More countries and key terms can be added here
    }

    university_name = university_name.lower()

    # Check in specific universities list
    for uni, country in specific_universities.items():
        if uni.lower() in university_name:
            return country

    # Infer from key terms
    for country, keywords in country_inference.items():
        if any(keyword.lower() in university_name for keyword in keywords):
            return country

    # Default to 'Unknown' if no match is found
    return "Unknown"

In [ ]:
# Identify the country of university associated with the authors
oth['Country'] = oth['University'].apply(get_country_from_university_comprehensive)
oth = oth.sort_values(by = 'Country')

In [ ]:
# Distinguish authors with known/unknown country of associated university

known = oth[oth['Country']!= 'Unknown']
unknown = oth[(oth['Country']== 'Unknown')& (~oth['University'].isna())].sort_values(by = "University")
unknown = unknown[unknown['University'].apply(lambda x: isinstance(x, str) and not x.strip().isdigit())]
unknown_ref = unknown[unknown['University'].str.strip() != ""]

In [ ]:
known

,Authors,University,Source,Last Name,First Name,Middle Name,Country
2485,Frank decker,"Honorary associate, the university of sydney l...",Cepr,Decker,Frank,NaN,Australia
213,Sergey alexeev,Senior research fellow research fellow univers...,Cepr,Alexeev,Sergey,NaN,Australia
138,Reshad n ahsan,"Senior lecturer, department of economics unive...",Cepr,Ahsan,Reshad,N,Australia
1571,Thomas ruf,"university of new south wales, school of bank...",Afa,Ruf,Thomas,NaN,Australia
1611,Ravindra sastry,"university of melbourne, department of finance",Afa,Sastry,Ravindra,NaN,Australia
...,...,...,...,...,...,...,...
522,Manuel fernandez,imt dubai,Afa,Fernandez,Manuel,NaN,United Arab Emirates
1544,Noor ulain rizvi,manipal university dubai,Afa,Rizvi,Noor,Ulain,United Arab Emirates
3609,Nicklas garemo,"Director mckinsey & company, abu dhabi",Cepr,Garemo,Nicklas,NaN,United Arab Emirates
1423,Youguo liang,abu dhabi investment authority,Afa,Liang,Youguo,NaN,United Arab Emirates


Note that the university column is not fully cleaned. 

It includes position + associated instiution. 

Thus, we should further clean this column to separate the position and the associate insitute. This will give us higher numbers of asscoatied authors per institution for larger institutions. 

In [ ]:
# Number of authors based on each country. 

oth.groupby('Country').agg('count').sort_values(by = 'University',ascending = False)

,Authors,University,Source,Last Name,First Name,Middle Name
Country,,,,,,
USA,4391,4391,4391,4391,4391,1322
Unknown,800,791,800,800,800,298
Canada,457,457,457,457,457,112
UK,161,161,161,161,161,22
Australia,87,87,87,87,87,24
France,81,81,81,81,81,9
Singapore,70,70,70,70,70,18
Spain,68,68,68,68,68,47
China,65,65,65,65,65,4


In [ ]:
# Save faculty profile

AFA =  oth.sort_values(by = "Last Name")
AFA.to_excel("Author_Profile.xlsx")